In [1]:
import os
import time as timer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import open3d as o3d
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

from load_3d_data import load_data
from visualization_utils import visualize
import train_facetalk_utils as tu
import vae

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define Constants

In [2]:
DATA_DIR = './data'
MODEL_DIR = './models'

EPOCHS = 100
BATCH_SIZE = 16
LEARNING_RATE = 1e-3
dh = 5000
dz = 500
beta = 0.75

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Helper Functions

In [3]:
"""
Utility functions
"""
def reshape_vertices(inversed):
    """
    Function to reshape the inversed principal components
    to vertices
    """
    vertices = []
    for i in range(len(inversed)):
        vert = np.reshape(inversed[i], (-1, 3))
        vertices.append(vert)
    
    return vertices

def vertices_to_meshes(vertices, ori_meshes):
    """
    Function to convert vertices to meshes
    using triangles from original meshes
    """
    meshes = []
    for i in range(len(vertices)):
        triangles = np.asarray(ori_meshes[i].triangles)
        mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(vertices[i]), 
            o3d.utility.Vector3iVector(triangles))
        mesh.compute_vertex_normals()
        mesh.compute_triangle_normals()
        mesh.paint_uniform_color([0.5, 0.5, 0.5])
        mesh.normalize_normals()
        meshes.append(mesh)
    return meshes

## Data Preparation

In [4]:
dirname = './data/FaceTalk'
files, expressions = load_data(dirname)
len(files)

1183

In [5]:
"""
Split to train and test set (90:10)
"""
X_train, X_test = train_test_split(files, test_size=.1, random_state=42)
len(X_train), len(X_test)

(1064, 119)

### Get Vertices from 3D Mesh

In [6]:
def get_vertices(files):
    """
    Function to get each 3d mesh file's vertices
    """
    vertices = []
    for file in files:
        vert = np.asarray(file.vertices)
        vert_reshaped = vert.reshape([1, -1])[0]
        vertices.append(vert_reshaped)
    return np.asarray(vertices)

In [7]:
X_train_v = get_vertices(X_train).reshape(-1, 5023, 3)
X_test_v = get_vertices(X_test).reshape(-1, 5023, 3)
X_train_v.shape, X_test_v.shape

((1064, 5023, 3), (119, 5023, 3))

In [8]:
train_dl = DataLoader(X_train_v, batch_size=BATCH_SIZE)
test_dl = DataLoader(X_test_v, batch_size=BATCH_SIZE)

In [9]:
[n, dx1, dx2] = X_train_v.shape

## Train Variational Autoencoder

In [10]:
print(f"Device: {DEVICE}")

Device: cuda


In [12]:
model = vae.VAE_FT(d_in=dx1*dx2, d_z=dz, d_h=dh).to(DEVICE)
print(model)

# Persistent file to store the model
model_path = os.path.join(MODEL_DIR, 'vae_ep100_facetalk.pth')

VAE_FT(
  (encoder): Sequential(
    (0): Linear(in_features=15069, out_features=5000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5000, out_features=1000, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=500, out_features=5000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5000, out_features=15069, bias=True)
  )
)


In [13]:
"""Define loss function"""
loss_fn = tu.vae_loss

"""Define optimizer"""
optimizer = optim.Adam(
    model.parameters(), 
    lr=LEARNING_RATE
)

In [14]:
"""Train the Autoencoder"""

for t in range(EPOCHS):
    print(f"Epoch {t+1} out of {EPOCHS}\n ------------")
    
    start = timer.time()
    tu.train_vae(train_dl, model, loss_fn, optimizer, beta=beta)
    elapsed_time = timer.time() - start # this timing method ONLY works for CPU computation, not for GPU/cuda calls
    print(f" > Training time: {elapsed_time:>.2f} seconds")
    
    test_loss = tu.test_vae(test_dl, model, loss_fn, beta=beta)
    print(f" > Test reconstruction loss: {test_loss:>.2f}")
    
    # Save model
    torch.save(model.state_dict(), model_path)
    print(f"Model {model_path} stored!")
    
print("Done!")

Epoch 1 out of 100
 ------------
Loss: 13803.972656 [    0]/ 1064
 > Training time: 5.30 seconds
 > Test reconstruction loss: 800.74
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 2 out of 100
 ------------
Loss: 918.820129 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 705.93
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 3 out of 100
 ------------
Loss: 805.175293 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 600.77
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 4 out of 100
 ------------
Loss: 710.547974 [    0]/ 1064
 > Training time: 3.60 seconds
 > Test reconstruction loss: 414.62
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 5 out of 100
 ------------
Loss: 565.913086 [    0]/ 1064
 > Training time: 3.60 seconds
 > Test reconstruction loss: 379.20
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 6 out of 100
 ------------
Loss: 514.505493 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruc

 > Training time: 3.59 seconds
 > Test reconstruction loss: 137.84
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 48 out of 100
 ------------
Loss: 178.112717 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 209.72
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 49 out of 100
 ------------
Loss: 228.388611 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 308.03
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 50 out of 100
 ------------
Loss: 331.046204 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 138.84
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 51 out of 100
 ------------
Loss: 150.845810 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 346.05
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 52 out of 100
 ------------
Loss: 536.210083 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 241.97
Model ./models\vae_ep100_facetalk.pth store

Model ./models\vae_ep100_facetalk.pth stored!
Epoch 94 out of 100
 ------------
Loss: 204.441040 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 179.54
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 95 out of 100
 ------------
Loss: 195.818207 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 184.62
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 96 out of 100
 ------------
Loss: 204.154419 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 244.29
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 97 out of 100
 ------------
Loss: 265.020355 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 186.75
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 98 out of 100
 ------------
Loss: 253.119186 [    0]/ 1064
 > Training time: 3.59 seconds
 > Test reconstruction loss: 201.56
Model ./models\vae_ep100_facetalk.pth stored!
Epoch 99 out of 100
 ------------
Loss: 218.316467 [    0]/ 1064